<a href="https://colab.research.google.com/github/zmiern/ie423/blob/main/task3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Task 3
###Predicting  customer purchase amount using Black Friday sales dataset.

## Initialize

In [175]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from google.colab import drive
drive.mount('/content/drive')

import warnings
warnings.filterwarnings('ignore')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Load Data

In [117]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [118]:
dfMlb = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/train.csv')

dfMlb.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN,1057
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,7969


In [119]:

dfMlb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550068 entries, 0 to 550067
Data columns (total 12 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   User_ID                     550068 non-null  int64  
 1   Product_ID                  550068 non-null  object 
 2   Gender                      550068 non-null  object 
 3   Age                         550068 non-null  object 
 4   Occupation                  550068 non-null  int64  
 5   City_Category               550068 non-null  object 
 6   Stay_In_Current_City_Years  550068 non-null  object 
 7   Marital_Status              550068 non-null  int64  
 8   Product_Category_1          550068 non-null  int64  
 9   Product_Category_2          376430 non-null  float64
 10  Product_Category_3          166821 non-null  float64
 11  Purchase                    550068 non-null  int64  
dtypes: float64(2), int64(5), object(5)
memory usage: 50.4+ MB


We need to see the the cells which includes missing values.

In [120]:
dfMlb.isna().sum()

User_ID                            0
Product_ID                         0
Gender                             0
Age                                0
Occupation                         0
City_Category                      0
Stay_In_Current_City_Years         0
Marital_Status                     0
Product_Category_1                 0
Product_Category_2            173638
Product_Category_3            383247
Purchase                           0
dtype: int64

We need to see which columnns include string values and which include numerical values.

In [121]:
string_columns = dfMlb[dfMlb.columns.tolist()].select_dtypes(include='object').columns.tolist()
numerical_columns = [col for col in dfMlb.columns if dfMlb[col].dtype in ['int64', 'float64']]
print("String Columns: {string_columns}".format(string_columns=string_columns))
print("Numerical Columns: {numerical_columns}".format(numerical_columns=numerical_columns))

String Columns: ['Product_ID', 'Gender', 'Age', 'City_Category', 'Stay_In_Current_City_Years']
Numerical Columns: ['User_ID', 'Occupation', 'Marital_Status', 'Product_Category_1', 'Product_Category_2', 'Product_Category_3', 'Purchase']


To have a better understanding of data, we need to have many unique attribues each column has. Later we can convert them to int values.


In [122]:
for col in dfMlb.columns:
  print("Column:", col)
  print("Unique Values:", dfMlb[col].unique())
  print("Number of Unique Values:", len(dfMlb[col].unique()))
  print("----")

Column: User_ID
Unique Values: [1000001 1000002 1000003 ... 1004113 1005391 1001529]
Number of Unique Values: 5891
----
Column: Product_ID
Unique Values: ['P00069042' 'P00248942' 'P00087842' ... 'P00370293' 'P00371644'
 'P00370853']
Number of Unique Values: 3631
----
Column: Gender
Unique Values: ['F' 'M']
Number of Unique Values: 2
----
Column: Age
Unique Values: ['0-17' '55+' '26-35' '46-50' '51-55' '36-45' '18-25']
Number of Unique Values: 7
----
Column: Occupation
Unique Values: [10 16 15  7 20  9  1 12 17  0  3  4 11  8 19  2 18  5 14 13  6]
Number of Unique Values: 21
----
Column: City_Category
Unique Values: ['A' 'C' 'B']
Number of Unique Values: 3
----
Column: Stay_In_Current_City_Years
Unique Values: ['2' '4+' '3' '1' '0']
Number of Unique Values: 5
----
Column: Marital_Status
Unique Values: [0 1]
Number of Unique Values: 2
----
Column: Product_Category_1
Unique Values: [ 3  1 12  8  5  4  2  6 14 11 13 15  7 16 18 10 17  9 20 19]
Number of Unique Values: 20
----
Column: Produ

'Gender', 'Age', 'City_Category', 'Stay_In_Current_City_Years' are categorical values which have non-integer values. We change them using LabelEncoder.

In [123]:
#Gender
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
dfMlb['Gender'] = label_encoder.fit_transform(dfMlb['Gender'])
print(dfMlb.Gender)
print(dfMlb.Gender.unique())


0         0
1         0
2         0
3         0
4         1
         ..
550063    1
550064    0
550065    0
550066    0
550067    0
Name: Gender, Length: 550068, dtype: int64
[0 1]


In [124]:
# Age
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
dfMlb['Age'] = label_encoder.fit_transform(dfMlb['Age'])
print(dfMlb.Age)
print(dfMlb.Age.unique())

0         0
1         0
2         0
3         0
4         6
         ..
550063    5
550064    2
550065    2
550066    6
550067    4
Name: Age, Length: 550068, dtype: int64
[0 6 2 4 5 3 1]


In [125]:
#City Category
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
dfMlb['City_Category'] = label_encoder.fit_transform(dfMlb['City_Category'])
print(dfMlb.City_Category)
print(dfMlb.City_Category.unique())

0         0
1         0
2         0
3         0
4         2
         ..
550063    1
550064    2
550065    1
550066    2
550067    1
Name: City_Category, Length: 550068, dtype: int64
[0 2 1]


In [126]:
#City Stay_In_Current_City_Years
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
dfMlb['Stay_In_Current_City_Years'] = label_encoder.fit_transform(dfMlb['Stay_In_Current_City_Years'])
print(dfMlb.Stay_In_Current_City_Years)
print(dfMlb.Stay_In_Current_City_Years.unique())

0         2
1         2
2         2
3         2
4         4
         ..
550063    1
550064    3
550065    4
550066    2
550067    4
Name: Stay_In_Current_City_Years, Length: 550068, dtype: int64
[2 4 3 1 0]


There are too many unique data in User ID. I will category them accordance with their purchase amount.

In [200]:

for col in dfMlb.columns:
  print("Column:", col)
  print("Unique Values:", dfMlb[col].unique())
  print("Number of Unique Values:", len(dfMlb[col].unique()))
  print("----")

Column: User_ID
Unique Values: [1000001 1000002 1000003 ... 1004113 1005391 1001529]
Number of Unique Values: 5891
----
Column: Gender
Unique Values: [0 1]
Number of Unique Values: 2
----
Column: Age
Unique Values: [0 6 2 4 5 3 1]
Number of Unique Values: 7
----
Column: Occupation
Unique Values: [10 16 15  7 20  9  1 12 17  0  3  4 11  8 19  2 18  5 14 13  6]
Number of Unique Values: 21
----
Column: City_Category
Unique Values: [0 2 1]
Number of Unique Values: 3
----
Column: Stay_In_Current_City_Years
Unique Values: [2 4 3 1 0]
Number of Unique Values: 5
----
Column: Marital_Status
Unique Values: [0 1]
Number of Unique Values: 2
----
Column: Product_Category_1
Unique Values: [ 3  1 12  8  5  4  2  6 14 11 13 15  7 16 18 10 17  9 20 19]
Number of Unique Values: 20
----
Column: Product_Category_2
Unique Values: [nan  6. 14.  2.  8. 15. 16. 11.  5.  3.  4. 12.  9. 10. 17. 13.  7. 18.]
Number of Unique Values: 18
----
Column: Product_Category_3
Unique Values: [nan 14. 17.  5.  4. 16. 15.  

## Build Random Forest Model

In [128]:

y = dfMlb.loc[:,['Purchase']].values.ravel()
X = dfMlb.drop(['Purchase'],axis=1)

In [129]:

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.8, test_size=0.2,random_state=42)

In [130]:
from sklearn.ensemble import RandomForestRegressor

# Function for building and scoring Random Forest models
def get_random_forest_mae(X_trn, X_tst, y_trn, y_tst):
    mdlRfsMlb = RandomForestRegressor(random_state=42)
    mdlRfsMlb.fit(X_trn, y_trn)
    y_tst_prd = mdlRfsMlb.predict(X_tst)
    mae = mean_absolute_error(y_tst, y_tst_prd)
    return (mae)

In [131]:
# Select numeric features
cols_num = [col for col in X.columns if X[col].dtype in ['int64', 'float64']]
Xnum = X[cols_num]

# Split numeric features into training and test sets
Xnum_train, Xnum_test, y_train, y_test = train_test_split(Xnum,y,train_size=0.8, test_size=0.2,random_state=42)

In [132]:
# Try to build a model using all numeric features
get_random_forest_mae(Xnum_train, Xnum_test, y_train, y_test)

ValueError: Input X contains NaN.
RandomForestRegressor does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

Seems like some of the numeric columns have **Missing Values**.

In [199]:
# Count number of missing values in each column of the training data
Xnum_train.isna().sum()

User_ID                            0
Gender                             0
Age                                0
Occupation                         0
City_Category                      0
Stay_In_Current_City_Years         0
Marital_Status                     0
Product_Category_1                 0
Product_Category_2            139044
Product_Category_3            306828
dtype: int64

#### Approach 1. Drop columns with missing values
The simplest option is to **drop columns** with missing values.

In [100]:
# Identify columns with missing values and then drop such columns
cols_num_null = [col for col in Xnum_train.columns
    if Xnum_train[col].isnull().any()]
Xnum_train_drpnull = Xnum_train.drop(cols_num_null, axis=1)
Xnum_test_drpnull = Xnum_test.drop(cols_num_null, axis=1)

In [101]:
print('MAE from Approach 1 (Drop features with missing values):')
print(get_random_forest_mae(Xnum_train_drpnull, Xnum_test_drpnull, y_train, y_test))

MAE from Approach 1 (Drop features with missing values):
2099.0747686640457


#### Approach 2. Fill missing values by Imputation
**Imputation** fills in the missing values with some number. For instance, we can fill in the mean value along each column.

In [133]:
# Replace with mean value
Xnum_train_repnull = Xnum_train.fillna(Xnum_train.mean())
Xnum_test_repnull = Xnum_test.fillna(Xnum_test.mean())

print('MAE from Approach 2 (Replace missing values with mean):')
print(get_random_forest_mae(Xnum_train_repnull, Xnum_test_repnull, y_train, y_test))

MAE from Approach 2 (Replace missing values with mean):
2158.7366138414022


In [134]:
# Going forward, let us replace all missing numeric values with the column mean
X_train[cols_num]=Xnum_train_repnull[cols_num]
X_test[cols_num]=Xnum_test_repnull[cols_num]

Dropping columns that have missing values actually resulted with better mae. This may be resulted from high variance among the dataset. I will drop respective columns.


In [197]:
dfMlb_copy = dfMlb.drop(columns=['Product_Category_2','Product_Category_3'])

utexas_ds_orie_divider_gray.png

## Build Gradient Boosted Tree Model

In [204]:
from xgboost import XGBRegressor
Xle_train = X_train.copy()
Xle_test = X_test.copy()
#Build and score default Gradient Boosting Model
mdlXgbMlb = XGBRegressor()
mdlXgbMlb.fit(Xle_train[cols_num + cols_cat], y_train)
y_test_pred = mdlXgbMlb.predict(Xle_test[cols_num + cols_cat])
mae = mean_absolute_error(y_test_pred, y_test)

print("MAE from default XGBoost model:")
print(mae)

#Build and score a tuned Gradient Boosting Model
mdlXgbMlb = XGBRegressor(n_estimators=5000, learning_rate=0.01, max_depth=5)
mdlXgbMlb.fit(Xle_train[cols_num + cols_cat], y_train)
y_test_pred = mdlXgbMlb.predict(Xle_test[cols_num + cols_cat])
mae = mean_absolute_error(y_test_pred, y_test)

print("MAE from tuned XGBoost model:")
print(mae)

ValueError: feature_names must be unique. Duplicates found: ['Age', 'City_Category', 'Gender', 'Marital_Status', 'Product_Category_1', 'Product_Category_2', 'Product_Category_3', 'Stay_In_Current_City_Years']

Hold on. The result here is worse than what obtained from our previous Random Forest model.  Let's try to improve this by **tuning the parameters** that drive the Gradient Boosting model.  Below are some popular parameters...

`n_estimators`: maximum number of decision trees that will be ensembled

`max_depth`: maximum depth of each tree (typically 3-10)

`learning_rate`: weight applied to each tree (typically 0.01-0.2)

In [202]:
#Build and score a tuned Gradient Boosting Model
mdlXgbMlb = XGBRegressor(n_estimators=5000, learning_rate=0.01, max_depth=5)
mdlXgbMlb.fit(Xle_train[cols_num + cols_cat], y_train)
y_test_pred = mdlXgbMlb.predict(Xle_test[cols_num + cols_cat])
mae = mean_absolute_error(y_test_pred, y_test)

print("MAE from tuned XGBoost model:")
print(mae)

NameError: name 'Xle_train' is not defined

Using a gradient boosted model, we're able to shave off another \$16K off the best MAE so far. Therefore, intelligently selecting trees to add to the ensemble using gradient descent combined with proper parameter tuning helps to significantly improve the result.

## Takeaways

* Expanded the model to include other Numerical features, and replaced missing values by *Imputation*
* Included Categorical features, and converted them to usable information by *Label Encoding*
* Ensembled many decision trees more intelligently using the *Gradient Boosting* model and *Parameter Tuning* for better results

utexas_ds_orie_divider_gray.png